In [639]:
import ROOT
import numpy as np
import pandas as pd
h   = 6.62607015e-34  # Planck's constant [J*s]
m_e = 9.10938356e-31  # Electron rest mass [kg]
q   = 1.60217663e-19  # Elementary charge [C]

def fit_lin(x, sx, y, sy, file_name="fit",title_name="fit", visual_scale_factor_x=1, visual_scale_factor_y=1):
    ROOT.gStyle.SetOptFit(1)
    graph = ROOT.TGraphErrors(len(x))
    for i in range(len(x)):
        graph.SetPoint(i, x[i], y[i])
        graph.SetPointError(i, sx[i], sy[i])

    fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
    fit_func.SetParameters(0, 1)  # initial guesses

    graph.Fit(fit_func)
    cal_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

    starting_point=0
    if visual_scale_factor_x != 1 or visual_scale_factor_y !=1:
        print(f"Applying a visual scaling factor of {visual_scale_factor_x} to x and {visual_scale_factor_y} to x and from data points{starting_point+1}.")
        for i in range(starting_point,len(x)):
            graph.SetPointError(i, sx[i] * visual_scale_factor_x, sy[i] * visual_scale_factor_y)
    graph.SetMarkerStyle(20)
    graph.SetMarkerColor(ROOT.kBlue)
    graph.SetTitle(title_name)
    graph.GetYaxis().RotateTitle(0) 
    # Move it slightly left so it doesn't hit the numbers
    graph.GetYaxis().SetTitleOffset(1.5)

    c = ROOT.TCanvas("c", "Line Fit", 800, 600)
    graph.Draw("AP")
    fit_func.Draw("same")
    c.Update()
    stats = graph.GetListOfFunctions().FindObject("stats")
    if stats:
        stats.SetX1NDC(0.15)  # lower-left x (0 to 1)
        stats.SetY1NDC(0.75)  # lower-left y
        stats.SetX2NDC(0.45)  # upper-right x
        stats.SetY2NDC(0.87)   # upper-right y
        stats.SetTextSize(0.03)
        c.Modified()

    ROOT.gPad.SetTicks(1, 1)
    ROOT.gPad.SetGrid(1, 1)
    c.Update()
    c.SaveAs(f"figures/{file_name}.pdf")
    del c, graph, fit_func
    return cal_parameters

def fit_plot(x,sx,y,sy,file_name="fit",title_name="fit", visual_scale_factor_x=1, visual_scale_factor_y=1):
    ROOT.gStyle.SetOptFit(1)
    graph = ROOT.TGraphErrors(len(x))
    for i in range(len(x)):
        graph.SetPoint(i, x[i], y[i])
        graph.SetPointError(i, sx[i], sy[i])

    starting_point=0
    if visual_scale_factor_x != 1 or visual_scale_factor_y !=1:
        print(f"Applying a visual scaling factor of {visual_scale_factor_x} to x and {visual_scale_factor_y} to x and from data points{starting_point+1}.")
        for i in range(starting_point,len(x)):
            graph.SetPointError(i, sx[i] * visual_scale_factor_x, sy[i] * visual_scale_factor_y)
    graph.SetMarkerStyle(20)
    graph.SetMarkerColor(ROOT.kBlue)
    graph.SetTitle(title_name)
    graph.GetYaxis().RotateTitle(0) 
    # Move it slightly left so it doesn't hit the numbers
    graph.GetYaxis().SetTitleOffset(1.5)

    c = ROOT.TCanvas("c", "Line Fit", 800, 600)
    graph.Draw("AP")
    c.Update()

    ROOT.gPad.SetTicks(1, 1)
    ROOT.gPad.SetGrid(1, 1)
    c.Update()
    c.SaveAs(f"figures/{file_name}.pdf")
    del c, graph


first let us calcualte debrogie wavelength from: $$\lambda=\frac{h}{\sqrt{2q_em_eV}}$$  $$\sigma_{\lambda}=\frac{h\sigma_V}{\sqrt{8q_em_eV^3}}=\lambda\frac{\sigma_V}{2V}$$

In [640]:
def get_lambda(V,V_s=None):

    coeff_meters = h / np.sqrt(2 * m_e * q)

    coeff_nm = coeff_meters * 1.0e9
    lambda_nm=[]
    for i in V:
        lambda_nm.append(float(coeff_nm / (np.sqrt(i))))

    if V_s is not None:
        d_uncertainty_nm=[]
        for i in range(len(V_s)):
            d_uncertainty_nm.append(float(lambda_nm[i] * (V_s[i] /(2*V[i]))))
        return lambda_nm,d_uncertainty_nm

    return lambda_nm

and angel calculaton from arch length s follows: $$r = R \sin\left(\frac{s}{R}\right)$$
$$r_2 = R \cos\left(\frac{s}{R}\right)$$
  $$\theta = \frac{1}{2} \arctan\left(\frac{r}{L-(r-r_2)}\right)$$

\begin{equation}
    \sigma_{\theta} = \sqrt{ \left( \frac{\partial \theta}{\partial s} \sigma_s \right)^2 + \left( \frac{\partial \theta}{\partial L} \sigma_L \right)^2 + \left( \frac{\partial \theta}{\partial R} \sigma_R \right)^2 }
\end{equation}

In [641]:
def calculate_theta1(s_arc, s_err, L=14.0, L_err=0.3, R=4.3, R_err=0.1):
    """
    Vectorized version: Converts measured arc lengths to theta.
    Accepts radius.
    """
    s_arc = np.array(s_arc)
    s_err = np.array(s_err)
 
    # r = R * sin(s / (2*R))
    term = s_arc / (R)
    r = R * np.sin(term)
    
    # Derivatives for r
    dr_ds = 0.5 * np.cos(term)
    dr_dR = np.sin(term) - term * np.cos(term)
    
    # Error in r
    r_err = np.sqrt((dr_ds * s_err)**2 + (dr_dR * R_err)**2)

    # 2*theta = arctan(r / L)
    u = r / L
    two_theta = np.arctan(u)
    
    # Derivatives for 2*theta
    denom = 1 + u**2
    d2theta_dr = (1/L) / denom
    d2theta_dL = (-r/L**2) / denom
    
    # Error in 2*theta
    two_theta_err = np.sqrt((d2theta_dr * r_err)**2 + (d2theta_dL * L_err)**2)
    
    theta_rad = two_theta / 2
    theta_err = two_theta_err / 2
    
    return theta_rad, theta_err

In [642]:
def calculate_theta_precise(s_arc, s_err, L=14.0, L_err=0.3, R=4.3, R_err=0.1):
    """
    Calculates theta using the precise spherical geometry provided.
    Relations:
      r = R * sin(s/R)
      r2 = R * cos(s/R)
      theta = 0.5 * arctan(r / (L - r2))
    Parameters:
    s_arc  : Measured arc radius [cm]
    s_err  : Uncertainty in arc radius [cm]
    L      : Distance parameter [cm]
    L_err  : Uncertainty in L [cm]
    R      : Radius of curvature [cm]
    R_err  : Uncertainty in R [cm]
    """
    # 1. Vectorize inputs
    s = np.array(s_arc)
    sigma_s = np.array(s_err)

    # 2. Intermediate Variables
    phi = s / R
    sin_phi = np.sin(phi)
    cos_phi = np.cos(phi)
    r = R * sin_phi
    r2 = R * cos_phi
    
    # Denominator for the arctan argument
    denom = L - (R-r2)
    # The argument for arctan
    u = r / denom

    # theta = 0.5 * arctan(u)
    theta_rad = 0.5 * np.arctan(u)
    
    # --- Error Propagation Starts Here ---
    
    # Partial derivatives of phi
    dphi_ds = 1.0 / R
    dphi_dR = -s / (R**2)
    
    # Partial derivatives of r
    dr_ds = R * cos_phi * dphi_ds 
    dr_dR = sin_phi + R * cos_phi * dphi_dR 
    
    # Partial derivatives of r2 (l2)
    dr2_ds = -R * sin_phi * dphi_ds
    dr2_dR = cos_phi - R * sin_phi * dphi_dR
    
    # Partial derivatives of u = r / denom
    # du = (dr * denom - r * d(denom)) / denom^2
    # d(denom) = dL - dr2
    
    # du/ds
    d_denom_ds = -dr2_ds
    du_ds = (dr_ds * denom - r * d_denom_ds) / (denom**2)
    
    # du/dL
    d_denom_dL = 1.0
    du_dL = (0 * denom - r * d_denom_dL) / (denom**2) # = -r / denom^2
    
    # du/dR
    d_denom_dR = -dr2_dR
    du_dR = (dr_dR * denom - r * d_denom_dR) / (denom**2)
    
    # Partial derivatives of theta = 0.5 * arctan(u)
    dtheta_du = 0.5 / (1 + u**2)
    
    # Final Derivatives for Sigma
    dtheta_ds = dtheta_du * du_ds
    dtheta_dL = dtheta_du * du_dL
    dtheta_dR = dtheta_du * du_dR
  
    theta_err = np.sqrt(
        (dtheta_ds * sigma_s)**2 + 
        (dtheta_dL * L_err)**2 + 
        (dtheta_dR * R_err)**2
    )
    
    return theta_rad, theta_err

In [ ]:
sigma_V = 100.0 

# --- Helper Function to Prepare Data ---
def prepare_fit_data(df, sigma_V):
    """
    Extracts and transforms dataframe columns into vectors for the fit.
    X = 1/sqrt(V), Y = sin(theta)
    """

    V = df['Voltage (V)'].to_numpy()
    theta = df['theta_val_rad'].to_numpy()
    theta_err = df['theta_unc_rad'].to_numpy()
    
    x = 1.0 / np.sqrt(V)
    # Error Propagation for x = V^(-0.5)
    # sigma_x = x * 0.5 * (sigma_V / V)
    sx = x * 0.5 * (sigma_V / V)
    
    y = np.sin(theta)
    # Error Propagation for y = sin(theta)
    # sigma_y = cos(theta) * sigma_theta
    sy = np.cos(theta) * theta_err
    
    return x, sx, y, sy


In [644]:
# Load the dataset
df = pd.read_csv('Diffraction_Graphite.csv')
# accessing the first column 'Voltage (V)' and taking every 2nd value
voltage_list = df['Voltage (V)'][::2].tolist() #[::2]
print("Voltage Values:", voltage_list)
voltage_s_list=np.full(len(voltage_list), 100)
print(voltage_s_list)
y,sy=get_lambda(voltage_list,voltage_s_list)


Voltage Values: [2900, 3200, 3500, 3800, 3900, 4000, 4100, 4300, 4600, 5000]
[100 100 100 100 100 100 100 100 100 100]


In [645]:
fit_plot(y,sy,voltage_list,voltage_s_list,title_name="de Broglie Wavelength;#lambda(nm);Volts")

Info in <TCanvas::Print>: pdf file figures/fit.pdf has been created


In [646]:
df = pd.read_csv('Diffraction_Graphite.csv')
voltage_list = df['Voltage (V)'].tolist()
print("Voltage Values:", voltage_list)
voltage_s_list=np.full(len(voltage_list), 100)
df["lambda_nm"], df["lambda_unc_nm"] = get_lambda(voltage_list,voltage_s_list)
#radii_df = diams / 2.0
diams = df[['Inside Diameter (cm)', 'Outside Diameter (cm)', 'Inside Horizontal(cm)', 'Outside Horizontal (cm)']]

inst_unc = 0.1 
N = diams.shape[1] 
stat_unc_diam = diams.std(axis=1, ddof=1) / np.sqrt(N)
total_unc_diam = np.sqrt(stat_unc_diam**2 + inst_unc**2)


df['Radius_avg_cm'] = diams.mean(axis=1)
# ddof=1 ensures we calculate Sample Standard Deviation (N-1)
df['Radius_unc_cm'] = total_unc_diam
df["theta_val_rad"],df["theta_unc_rad"]=calculate_theta_precise(df['Radius_avg_cm'], df['Radius_unc_cm'])
print(df[['Voltage (V)', 'Ring Order', 'Radius_avg_cm', 'Radius_unc_cm','lambda_nm','lambda_unc_nm','theta_val_rad','theta_unc_rad']])

Voltage Values: [2900, 2900, 3200, 3200, 3500, 3500, 3800, 3800, 3900, 3900, 4000, 4000, 4100, 4100, 4300, 4300, 4600, 4600, 5000, 5000]
    Voltage (V)  Ring Order  Radius_avg_cm  Radius_unc_cm  lambda_nm  \
0          2900   1st Order           1.70       0.115470   0.022774   
1          2900   2nd Order           2.80       0.115470   0.022774   
2          3200   1st Order           1.45       0.119024   0.021680   
3          3200   2nd Order           2.55       0.144338   0.021680   
4          3500   1st Order           1.40       0.115470   0.020730   
5          3500   2nd Order           2.45       0.119024   0.020730   
6          3800   1st Order           1.35       0.119024   0.019895   
7          3800   2nd Order           2.35       0.119024   0.019895   
8          3900   1st Order           1.30       0.115470   0.019639   
9          3900   2nd Order           2.35       0.119024   0.019639   
10         4000   1st Order           1.35       0.119024   0.019391   

In [647]:
df

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_avg_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
0,2900,1st Order,1.6,1.8,1.6,1.8,0.42,0.022774,0.000393,1.70,0.115470,0.060286,0.003914
1,2900,2nd Order,2.7,2.9,2.7,2.9,0.42,0.022774,0.000393,2.80,0.115470,0.098047,0.003789
2,3200,1st Order,1.4,1.6,1.3,1.5,0.42,0.021680,0.000339,1.45,0.119024,0.051521,0.004073
3,3200,2nd Order,2.6,2.8,2.3,2.5,0.42,0.021680,0.000339,2.55,0.144338,0.089605,0.004434
4,3500,1st Order,1.3,1.5,1.3,1.5,0.42,0.020730,0.000296,1.40,0.115470,0.049762,0.003971
5,3500,2nd Order,2.4,2.6,2.3,2.5,0.42,0.020730,0.000296,2.45,0.119024,0.086202,0.003878
6,3800,1st Order,1.3,1.5,1.2,1.4,0.42,0.019895,0.000262,1.35,0.119024,0.048001,0.004094
7,3800,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019895,0.000262,2.35,0.119024,0.082786,0.003892
8,3900,1st Order,1.2,1.4,1.2,1.4,0.42,0.019639,0.000252,1.30,0.115470,0.046238,0.003990
9,3900,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019639,0.000252,2.35,0.119024,0.082786,0.003892


In [648]:
df.to_csv('processed_data.csv', index=False,float_format='%.4e')

we will fit $1/\sqrt{V}$ vs. $sin(\theta)$:
\begin{equation}
    2d\sin(\theta)=n\lambda=1\left(\frac{h}{\sqrt{2m_eq_eV}}\right)
\end{equation}    

\begin{equation}
    \underbrace{\sin(\theta)}_{y} = \underbrace{\left[ \frac{h}{2d\sqrt{2m_e q}} \right]}_{\text{Slope } m} \cdot \underbrace{\frac{1}{\sqrt{V}}}_{x}
\end{equation}

In [649]:
# 1. Load the data
df = pd.read_csv('processed_data.csv')

# 2. Clean the 'Ring Order' column (fix "2nd  Order" typo)
df['Ring Order'] = df['Ring Order'].str.replace('  ', ' ', regex=False)

# 3. Create separate DataFrames
df_1st = df[df['Ring Order'] == '1st Order'].copy()
df_2nd = df[df['Ring Order'] == '2nd Order'].copy()

# 5. Display to verify
df_1st.to_csv('processed_data1.csv', index=False,float_format='%.4e')
df_2nd.to_csv('processed_data2.csv', index=False,float_format='%.4e')

In [650]:
df_1st

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_avg_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
0,2900,1st Order,1.6,1.8,1.6,1.8,0.42,0.022774,0.000393,1.70,0.11547,0.060286,0.003914
2,3200,1st Order,1.4,1.6,1.3,1.5,0.42,0.021680,0.000339,1.45,0.11902,0.051521,0.004073
4,3500,1st Order,1.3,1.5,1.3,1.5,0.42,0.020730,0.000296,1.40,0.11547,0.049762,0.003971
6,3800,1st Order,1.3,1.5,1.2,1.4,0.42,0.019895,0.000262,1.35,0.11902,0.048001,0.004094
8,3900,1st Order,1.2,1.4,1.2,1.4,0.42,0.019639,0.000252,1.30,0.11547,0.046238,0.003990
10,4000,1st Order,1.3,1.5,1.2,1.4,0.42,0.019391,0.000242,1.35,0.11902,0.048001,0.004094
12,4100,1st Order,1.2,1.4,1.2,1.4,0.42,0.019154,0.000234,1.30,0.11547,0.046238,0.003990
14,4300,1st Order,1.2,1.4,1.1,1.3,0.42,0.018703,0.000217,1.25,0.11902,0.044474,0.004114
16,4600,1st Order,1.2,1.4,1.1,1.3,0.42,0.018083,0.000197,1.25,0.11902,0.044474,0.004114
18,5000,1st Order,1.1,1.3,1.1,1.3,0.42,0.017344,0.000173,1.20,0.11547,0.042708,0.004008


In [651]:
df_2nd

,Voltage (V),Ring Order,Inside Diameter (cm),Outside Diameter (cm),Inside Horizontal(cm),Outside Horizontal (cm),Filtering Voltage (A),lambda_nm,lambda_unc_nm,Radius_avg_cm,Radius_unc_cm,theta_val_rad,theta_unc_rad
1,2900,2nd Order,2.7,2.9,2.7,2.9,0.42,0.022774,0.000393,2.80,0.11547,0.098047,0.003789
3,3200,2nd Order,2.6,2.8,2.3,2.5,0.42,0.021680,0.000339,2.55,0.14434,0.089605,0.004434
5,3500,2nd Order,2.4,2.6,2.3,2.5,0.42,0.020730,0.000296,2.45,0.11902,0.086202,0.003878
7,3800,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019895,0.000262,2.35,0.11902,0.082786,0.003892
9,3900,2nd Order,2.3,2.5,2.2,2.4,0.42,0.019639,0.000252,2.35,0.11902,0.082786,0.003892
11,4000,2nd Order,2.3,2.5,2.3,2.5,0.42,0.019391,0.000242,2.40,0.11547,0.084496,0.003802
13,4100,2nd Order,2.2,2.4,2.1,2.3,0.42,0.019154,0.000234,2.25,0.11902,0.079356,0.003907
15,4300,2nd Order,2.2,2.4,2.1,2.3,0.42,0.018703,0.000217,2.25,0.11902,0.079356,0.003907
17,4600,2nd Order,2.2,2.4,2.1,2.3,0.42,0.018083,0.000197,2.25,0.11902,0.079356,0.003907
19,5000,2nd Order,2.0,2.2,2.0,2.2,0.42,0.017344,0.000173,2.10,0.11547,0.074188,0.003841


In [652]:
# --- 3. Prepare Data for Both Orders ---
x_1st, sx_1st, y_1st, sy_1st = prepare_fit_data(df_1st, sigma_V)
x_2nd, sx_2nd, y_2nd, sy_2nd = prepare_fit_data(df_2nd, sigma_V)

print("--- Fitting 1th Order ---")
params_1st = fit_lin(x_1st, sx_1st, y_1st, sy_1st, 
                     title_name="lienar fit for the first ring; 1/#sqrt{V};sin(#theta)",
                     file_name="1st Order(10)")
print("\nFit Results (Intercept, Slope):")
print(f"1st Order: {params_1st}")
print("\n--- Fitting 2nd Order ---")
params_2nd = fit_lin(x_2nd, sx_2nd, y_2nd, sy_2nd, 
                     title_name="lienar fit for the second ring; 1/#sqrt{V};sin(#theta)",
                     file_name="2nd Order(11)")

print("\nFit Results (Intercept, Slope):")
print(f"2nd Order: {params_2nd}")

--- Fitting 1th Order ---

Fit Results (Intercept, Slope):
1st Order: [(-0.00925291775822138, 0.016376108811855947), (3.567196773547114, 1.014282459969904)]

--- Fitting 2nd Order ---

Fit Results (Intercept, Slope):
2nd Order: [(0.005609039604029609, 0.016274008561705154), (4.843887112960725, 1.0119472540151855)]
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.51909
NDf                       =            8
Edm                       =  1.27416e-06
NCalls                    =           48
p0                        =  -0.00925292   +/-   0.0163761   
p1                        =       3.5672   +/-   1.01428     
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.62772
NDf                       =            8
Edm                       =  1.38591e-07
NCalls                    =           56
p0                        =   0.00560904   +/-   0.016274    
p1                        = 

Info in <TCanvas::Print>: pdf file figures/1st Order(10).pdf has been created
Info in <TCanvas::Print>: pdf file figures/2nd Order(11).pdf has been created


$$slope=\frac{h}{2d\sqrt{2m_e q}}$$
then latice constant d: 
$$d=\frac{h}{2\cdot slope\sqrt{2m_e q}}$$
and the uncertainty is:
$$\sigma_d=d \cdot \frac{slope}{\sigma_{slope}}

In [ ]:
def get_lattice_spacing(slope, slope_err=None):
    """
    Calculates lattice spacing d from the slope of sin(theta) vs 1/sqrt(V).
    
    Parameters:
    slope     : The slope m obtained from the linear fit [units: sqrt(V)]
    slope_err : The uncertainty in the slope from the fit (sigma_m)
    
    Returns:
    d_nm      : Lattice spacing in nanometers
    d_err_nm  : Uncertainty in lattice spacing in nanometers (if slope_err provided)
    """
    coeff_meters = h / np.sqrt(2 * m_e * q)
    coeff_nm = coeff_meters * 1.0e9
    d_nm = coeff_nm / (2 * slope)
    
    if slope_err is not None:
        d_err_nm = d_nm * (slope_err / slope)
        return d_nm, d_err_nm
    
    return d_nm

In [654]:
get_lattice_spacing(3.6)

np.float64(0.17033694127977803)

In [655]:
slope,slope_unc=params_1st[1]
d_val, d_unc = get_lattice_spacing(slope, slope_unc)

print(f"Lattice Constant d_10: {d_val:.4e} ± {d_unc:.4e} nm")
slope,slope_unc=params_2nd[1]
d_val, d_unc = get_lattice_spacing(slope, slope_unc)

print(f"Lattice Constant d_11: {d_val:.4e} ± {d_unc:.4e} nm")

Lattice Constant d_10: 1.7190e-01 ± 4.8878e-02 nm
Lattice Constant d_11: 1.2660e-01 ± 2.6447e-02 nm
